<a href="https://colab.research.google.com/github/Aimee-ARC/Inteligencia_Artificial/blob/main/Genetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Genetic Algorithm in Python "Traveling Salesman"



##Inisialization
We import the necessary libraries

We create an empty graph

We read the Map.txt file and add nodes and edges to the graph

The following code creates an empty graph, reads a file called Map.txt, and adds nodes and edges to the graph. The Map.txt file should contain information about the cities and the connections between them, including the time and cost of travel between each pair of cities.

In [2]:
import networkx as nx
import random
import copy

Cities = nx.Graph()

with open("Map.txt") as file:
    for line in file:
        nodes = line.split()
        Cities.add_edge(nodes[0], nodes[1], time=int(nodes[2]), cost=int(nodes[3]))

##the map:
##Amsterdam Berlin 364 235
##Amsterdam Cologne 120 40
##Amsterdam Brussels 105 48
##Berlin Frankfurt 232 125
##Berlin Amsterdam 364 235
##Cologne Frankfurt 120 40
##Cologne Amsterdam 120 40
##Frankfurt Paris 480 345
##Frankfurt Milan 454 240
##Frankfurt Cologne 120 40
##Frankfurt Berlin 232 125
##Brussels Paris 82 80
##Brussels Amsterdam 105 48
##London Paris 136 98
##London Brussels 136 98
##Paris Lyon 112 185
##Paris London 136 98
##Paris Brussels 82 80
##Paris Barcelona 390 400
##Paris Madrid 225 380
##Paris Frankfurt 480 345
##Lyon Paris 112 185
##Lyon Milan 176 180
##Lyon Barcelona 200 320
##Milan Frankfurt 454 240
##Milan Rome 168 125
##Milan Lyon 176 180
##Barcelona Paris 390 400
##Barcelona Lyon 200 320
##Barcelona Madrid 150 98
##Madrid Paris 225 380
##Madrid Barcelona 150 98

The fitness of a particular route is determined by the Fitness function. The cost and uniqueness of the route's nodes are combined to form the fitness value. The number of unique nodes in the route determines the uniqueness, whereas the cost is the reciprocal of the total of the cost of each edge in the route. By attempting to access the weight of each edge in the route, the function first determines whether the provided route is valid. A route yields a fitness of 0 if it is invalid. Following that, the function computes the cost and the total unique nodes, returning the sum as the fitness value.

In [3]:
def Fitness(route):

    for i in range(len(route)-1):
      try:
        edge = Cities[route[i]][route[i+1]]
      except:
        return 0

    cost = 1/sum(Cities[route[i]][route[i+1]]['cost'] for i in range(len(route)-1))

    unique_nodes = len(set(route)) 

    return cost + unique_nodes

The RandomRoute function generates a random route by starting at a random node and visiting its unvisited neighboring nodes until the total travel time exceeds 72 hours (4320 minutes). The function takes no parameters and returns a tuple containing the generated route, the total travel time and the total travel cost.

In [4]:
def RandomRoute():

    current = random.choice(list(Cities.nodes()))

    route = [current]
    time = 0
    cost = 0
    visited_nodes = set([current])

    while time < 4320:

        adjacent = Cities.neighbors(current)

        
        adjacent_not_visited = [n for n in adjacent if n not in visited_nodes]
        if not adjacent_not_visited:
            break

        next = random.choice(adjacent_not_visited)

        route.append(next)
        edge = Cities[current][next]
        time += edge['time']
        cost += edge['cost']

        current = next
        visited_nodes.add(current)

    return route, time, cost


Utilizing the RandomRoute() function and the Fitness() function, the generate_random_solution() function constructs a random route and assesses its fitness. The function's result is a dictionary including the route, time, cost, and fitness of the produced route.


In [5]:
def generate_random_solution():
    route, time, cost = RandomRoute()
    fitness = Fitness(route)
    individual_info = {'route': route, 'time': time, 'cost': cost, 'fitness': fitness}
    
    return individual_info

A population of random paths is created using the function generatePopulation. GeneratiomSize, which denotes the size of the intended population, is a required input parameter. The function uses the generate_random_solution function to generate a random path for each iteration as it loops over GeneratiomSize iterations. Each route developed is then added to the population list. The function then returns the population, which is a list of routes that were produced.

In [6]:
def generatePopulation(genSize):
  population = []
  for i in range(genSize):
      route = generate_random_solution()
      population.append(route)
  return population

When a list of routes is provided as input, this method outputs a new list of those same routes, sorted in descending order according to their fitness values. The key 'fitness' in the dictionary that describes each route can be used to retrieve the fitness value of each route. The built-in Python function sorted() is used to sort the data, with the reverse argument set to True.


In [7]:
def sortByFitnessDescending(routes):
    
    sorted_routes = sorted(routes, key=lambda x: x['fitness'], reverse = True)

    return sorted_routes

By adding the time and cost values of each edge between subsequent nodes in the route, this function takes a route as an input and determines the overall time and cost of that route. The function returns (0, 0) if there isn't a graph edge connecting two of the route's nodes. The function then produces a tuple that includes the route's total time and total cost.

In [8]:
def route_cost_time(route):
    total_time = 0
    total_cost = 0
    for i in range(len(route)-1):

      try:
        edge = Cities[route[i]][route[i+1]]
      except:
        return 0, 0

      total_time += edge['time']
      total_cost += edge['cost']
    return total_time, total_cost

By switching a section of their routes, the crossover function takes two individuals, route_1 and route_2, and creates two new individuals. The function selects a crossing point at random, then combines the tails of the old routes to form the new ones. Then, using the route_cost_time and fitness functions, it produces new individuals utilizing the new routes and determines their time, cost, and fitness. The function then returns the two fresh people. In genetic algorithms, this function is frequently employed to produce new people in the following generation.

In [9]:
def crossover(route_1, route_2):
    # Choose a random crossover point
    crossover_point = random.randint(1, len(route_1['route']) - 1)

    # Swap the tails of the routes to create the new routes
    new_route_1 = route_1['route'][:crossover_point] + route_2['route'][crossover_point:]
    new_route_2 = route_2['route'][:crossover_point] + route_1['route'][crossover_point:]

    # Create the new individuals with the new routes and calculate their time and cost
    new_individual_1 = {'route': new_route_1, 'time': 0, 'cost': 0, 'fitness': 0}
    new_individual_2 = {'route': new_route_2, 'time': 0, 'cost': 0, 'fitness': 0}
    for individual in (new_individual_1, new_individual_2):
        individual['time'], individual['cost'] = route_cost_time(individual['route'])
        individual['fitness'] = Fitness(individual['route'])

    return new_individual_1, new_individual_2


The next generation of individuals for a genetic algorithm is produced by the function generate_next_generation. It requires two inputs: the number of elite individuals to be preserved in the following generation and a mating pool of individuals. The elite people are first included in the upcoming generation. Following that, it applies crossover to two randomly chosen parents from the mating pool to produce two new children. Only if the children have a fitness value that is not zero are they added to the following generation. This operation is repeated by the function until the targeted population size is attained. The future generation of people is the function's output.


In [10]:
def generate_next_generation (matingpool, eliteSize):
    # Create the initial next generation with the elite individuals
    nextGen = sortByFitnessDescending(matingpool)[:eliteSize]

    # Generate new children until reaching the desired population size
    while len(nextGen) < len(matingpool):
        # Choose two parents randomly from the mating pool
        parent1, parent2 = random.sample(matingpool, 2)

        # Generate two new children by single point crossover
        child1, child2 = crossover(parent1, parent2)

        # Add the children to the next generation if they have a non-zero fitness
        if child1['fitness'] > 0:
            nextGen.append(child1)
        if child2['fitness'] > 0 and len(nextGen) < len(matingpool):
            nextGen.append(child2)

    return nextGen


By randomly reversing a segment of the route between two points with a probability determined by the mutationRate argument, this function modifies a route. To prevent directly altering the input route, it first creates a replica of it. The mutationRate is then used to determine if the mutation should be implemented. If the mutation is used, the route's middle part is reversed and two random places are chosen along the way. The time, cost, and fitness of the modified route are then calculated and returned. The function returns the original input route if the mutation is not implemented.







In [11]:
def apply_mutation(route, mutationRate):
    mutated = route.copy()
    if random.uniform(0, 1) < mutationRate:

        position1 = random.randint(0, len(route['route']) - 2)
        position2 = random.randint(position1 + 1, len(route['route']) - 1)

        mutated['time'] = 0
        mutated['cost'] = 0
        mutated['fitness'] = 0

        route['route'][position1:position2+1] = reversed(route['route'][position1:position2+1])

        mutated['time'], mutated['cost'] = route_cost_time(mutated)

        mutated['fitness'] = Fitness(mutated)

    return mutated

The function mutatePopulation applies a mutation operation with a specific probability to every member of a population of persons (each of whom represents a potential solution to a problem). The inverse_mutate function carries out the mutation procedure by picking two random locations in the person's chromosome and flipping the order of the cities between those locations. The mutatePopulation method applies the mutation, adds the modified person to a new generation, and then returns the mutated population.

By flipping the cities in the chromosome between two randomly selected sites, the inverse_mutate function really performs the mutation. It then recalculates the mutated individual's time, cost, and fitness and returns the results.

In [12]:
def mutatePopulation(generation, mutationRate):
    mutatedGen = []

    for i in range(0, len(generation)):
        if random.uniform(0, 1) < mutationRate:
            apply_mutation = inverse_mutate(generation[i])
        else:
            apply_mutation = generation[i].copy()
        mutatedGen.append(apply_mutation)

    return mutatedGen

def inverse_mutate(route):
    mutated = route.copy()

    # Choose two random positions in the chromosome
    position1 = random.randint(0, len(route['route']) - 2)
    position2 = random.randint(position1 + 1, len(route['route']) - 1)

    # Reverse the order of the cities between the two chosen positions
    mutated['route'][position1:position2+1] = reversed(route['route'][position1:position2+1])

    # Recalculate time, cost and fitness for the new individual
    mutated['time'], mutated['cost'] = route_cost_time(mutated)
    mutated['fitness'] = Fitness(mutated)

    return mutated


This function executes the entire process of creating the next generation for a genetic algorithm and is a combination of the three previous methods (sortByFitnessDescending, generate_next_generation, and mutatePopulation).

The first step in the function is to rank the present generation according to fitness level and then pick the top candidates. Then, using a predetermined mutation rate, it breeds the remaining individuals to produce new offspring. The function then returns the following generation, which was created by combining the elite individuals and the altered offspring.


In [13]:
def breed_mutate_next_generation(currentGen, eliteSize, mutationRate):
    # Sort current generation by fitness
    Ranked = sortByFitnessDescending(currentGen)
    
    # Select elite individuals
    elite = Ranked[:eliteSize]
    
    # Breed the remaining individuals
    children = generate_next_generation (Ranked[eliteSize:], eliteSize)
    
    # Mutate the children
    mutatedChildren = mutatePopulation(children, mutationRate)
    
    # Combine the elite with the mutated children to form the next generation
    nextGen = elite + mutatedChildren
    
    return nextGen


This function applies a genetic algorithm to a route-related problem. It begins by creating a population of routes and then iterates through the number of generations indicated by the 'generations' option. The function chooses the most fit individuals from the present population (the "elite"), breeds new individuals by fusing the genes of the elite, mutates some of these new individuals, and then creates the following generation by fusing the elite with the mutant offspring. Natural selection, genetic recombination, and genetic mutation are the three processes that are used to choose the elite, breed, and modify organisms.

A fitness function, which in this case calculates the cost or distance of a specific route, establishes an individual's level of fitness. The best and worst routes discovered in each generation are recorded by the function, which publishes them to the console for monitoring. The function then returns the best route discovered for the entire run after processing the last generation.


**popSize** refers to the size of the initial population. That is, how many random individuals are generated at the start of the algorithm.

**eliteSize** refers to the number of elite individuals that will be selected from the current population to pass on to the next generation without modification. This is done to maintain genetic diversity and conserve the most fit individuals.

**mutationRate** refers to the mutation rate that applies to individuals in the population. The mutation rate determines the probability that a mutation will apply to an individual.

**generations** refers to the number of generations to be executed in the algorithm. Each generation produces a new population of individuals through selection, crossing, and mutation of the previous population.

In [14]:
def geneticAlgorithm(popSize, eliteSize, mutationRate, generations):
    pop = generatePopulation(popSize)
    bestRoute = sortByFitnessDescending(pop)[0]
    print(f"Initial best route distance: {bestRoute['fitness']}")
    print(f"Initial best route: {bestRoute['route']}")
    print("----------------------------------------")
    
    for i in range(generations):
        pop = breed_mutate_next_generation(pop, eliteSize, mutationRate)
        bestRoute = sortByFitnessDescending(pop)[0]
        worstRoute = sortByFitnessDescending(pop)[-1]
        print(f"Generation {i+1} - Best route distance: {bestRoute['fitness']}")
        print(f"Generation {i+1} - Best route: {bestRoute['route']}")
        print(f"Generation {i+1} - Worst route distance: {worstRoute['fitness']}")
        print(f"Generation {i+1} - Worst route: {worstRoute['route']}")
        print("----------------------------------------")
        
    print(f"Final best route distance: {sortByFitnessDescending(pop)[0]['fitness']}")
    print(f"Final best route: {sortByFitnessDescending(pop)[0]['route']}")
    return sortByFitnessDescending(pop)[0]
best_route = geneticAlgorithm(popSize=100, eliteSize=20, mutationRate=0.1, generations=20)


Initial best route distance: 11.000636132315522
Initial best route: ['Madrid', 'Barcelona', 'Lyon', 'Paris', 'London', 'Brussels', 'Amsterdam', 'Berlin', 'Frankfurt', 'Milan', 'Rome']
----------------------------------------
Generation 1 - Best route distance: 11.000636132315522
Generation 1 - Best route: ['Madrid', 'Barcelona', 'Lyon', 'Paris', 'London', 'Brussels', 'Amsterdam', 'Berlin', 'Frankfurt', 'Milan', 'Rome']
Generation 1 - Worst route distance: 0
Generation 1 - Worst route: ['Barcelona', 'Paris', 'Frankfurt', 'Berlin', 'Amsterdam', 'Brussels', 'London']
----------------------------------------
Generation 2 - Best route distance: 11.000636132315522
Generation 2 - Best route: ['Madrid', 'Barcelona', 'Lyon', 'Paris', 'London', 'Brussels', 'Amsterdam', 'Berlin', 'Frankfurt', 'Milan', 'Rome']
Generation 2 - Worst route distance: 0
Generation 2 - Worst route: ['Madrid', 'Paris', 'Brussels', 'London']
----------------------------------------
Generation 3 - Best route distance: 11.0